<a href="https://colab.research.google.com/github/Seongwoong-sk/PyTorch-Deep-Learning-Zero-To-All/blob/main/lab_11_3_longseq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 'longseq' example
**Longseq introduction**  
- want to use longer dataset
- but we wanna train in bigger chunkcs
- how can we create *fixed size sequence dataset* from long sentence?
  - 특정 사이즈의 winodw를 만들어서 하나씩 오른쪽으로 움직이면서 생성되는 chunk를 **input x**로 
  - 한 character만큼 오른쪽으로 shift한 글을 **y**로
  - `if you wan -> f you want` 이런식으로 반복되서 dataset 만듬
**Making seqeunce dataset from long sentence**    
**Adding FC layer and stacking RNN**    
**Code run through**


In [ ]:
import torch
import torch.optim as optim
import numpy as np

In [ ]:
# Random seed to make results deterministic and reproducible
torch.manual_seed(0)

In [ ]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

### make dictionary


In [ ]:
char_set = list(set(sentence))
char_dic = {c: i for i, c in enumerate(char_set)}

### hyper parameters


In [ ]:
dic_size = len(char_dic)
hidden_size = len(char_dic)

sequence_length = 10  # Any arbitrary number
learning_rate = 0.1

### data setting


In [ ]:
# x_data와 y_data를 담고자 하는 list 생성
x_data = []
y_data = []

# 앞에서부터 전체 문장의 길이 - length만큼 가져올 수 있는 chunk의 개수
# 움직이면서 sequence length 만큼의 chunk를 i부터 시작해서 sequence length의 길이 만큼의 chunk를 x_str로
# i + 1부터 시작해서 sequence length만큼 가지는 게 y_str
for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i : i+sequence_length]
    y_str = sentence[i + 1 : i+sequence_length+1]
    print(i, x_str, '->', y_str)

    # 위에서 구한 건 string이기 때문에 index로 구성된 list를 다음과 같이 character dictionary를 통해서 만들어 준 후 x_data, y_data라는 list에 넣어줌
    # 이렇게 되면 원하는 chunk들이 index로 변환된 list가 만들어짐
    x_data.append([char_dic[c] for c in x_str])  # x str to index
    y_data.append([char_dic[c] for c in y_str])  # y str to index

# One-hot vectorization
x_one_hot = [np.eye(dic_size)[x] for x in x_data]

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [ ]:
# transform as torch tensor variable

X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

### Adding FC layer and stacking RNN

In [ ]:
# declare RNN + FC
class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x


net = Net(dic_size, hidden_size, 2)

### loss & optimizer setting


In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

### start training


In [ ]:
for i in range(100):

    optimizer.zero_grad()

    outputs = net(X)

    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()


    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        # print(i, j, ''.join([char_set[t] for t in result]), loss.item())
        
        # 맨 처음에는 sequence length만큼 result에 있는 걸 전부 받아와서 sequence length만큼의 길이의 character를 만들고
        if j == 0:
            predict_str += ''.join([char_set[t] for t in result])

        # 그 이후부터는 나머지는 이제 맨 마지막 값 빼고는 모두 기본 거랑 겹치기 때문에 result의 맨 마지막 값만 가져와서 
        # 기존에 만들었던 predict_str에 하나씩 하나씩 붙임
        else:
            predict_str += char_set[result[-1]]

    print(predict_str)

a   uaauu auua auaua   uuauu  u  a   aa,u    a a saa  aa u aa uaaa aa  ua   u, sua a  uuauu ua u au uu  usuuu au aua  uu, u a uua aa u aa ua s  u  ua s asu uua uuuu aas a sua s au
                                                                                                                                                                                   
 nn  ony tttttttttttttttttt ttttttttttttttttttttttttttttttttttttttttttttttttttttottttttttttttttttttttttttottttttttttttttttttttttttttttttttttttttttttttttt tttttttottttttttttttttttt
                                                                                                                                                                                   
 rtttrtrttrttttttttrtrttrtrttrtrttrttt trttrttttrtrttttrtrtrtttttttttttttrtttttttrttrtrttrttttrttrtrtttttrtttrtrttttrttrttrtrttrttrttrtrttttttttt trtttttrtrttttrttrttrttrttrtrtttt
tuauauauauaauauauauaaaaaaaaaauauauauuauauauauauauauauauauaauauauaauauaaaaaaaaaaaauauaaaaauauaauaaaau